In [2]:
#!pip install torchio
#!pip install nilearn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
# import torchio
import os
import nilearn
from sklearn.model_selection import train_test_split
from skimage.transform import resize


%matplotlib inline

In [0]:
PATH_TO_MRI = './drive/My Drive/mri_data/Project/data/'
SCALE = [58, 70, 58]


In [0]:
#getting labels

targets = pd.read_csv('./drive/My Drive/mri_data/Project/Target.csv')
y = pd.get_dummies(targets, columns=['Gender'])['Gender_M'].values

In [0]:
#getting mri_data
imgs = []

for i, img_name in enumerate(sorted(os.listdir(PATH_TO_MRI))):
    img = nilearn.image.load_img(PATH_TO_MRI+img_name)
    imgs.append(resize(nilearn.image.get_data(img), SCALE))
    if i%25 == 0:
        print('{} iteration is finished.'.format(i))
    
X = np.stack(imgs)
del imgs #deleting for freeing space on disc
X.shape

In [0]:
class MriData(torch.utils.data.Dataset):
    def __init__(self, X, y):
        super(MriData, self).__init__()
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #may be that's bad idea to split using train_test_split. Instead of it - use indexes of data
#del X, y #deleting for freeing space on disc

In [0]:
train_dataset = MriData(X_train, y_train)
test_dataset = MriData(X_test, y_test)
#del X_train, X_test, y_train, y_test. #deleting for freeing space on disc

In [0]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=28, shuffle=True) 
val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=28, shuffle=False) 